In [2]:
from google.cloud import storage
import pandas as pd
import io
from io import BytesIO
import wget
import os, shutil
import time

In [3]:
base_dir = os.getcwd()
csv_dir = os.path.join(base_dir, 'img_csv')
data_dir = os.path.join(base_dir, 'data')

if os.path.isdir(data_dir):
    shutil. rmtree(data_dir)
os.mkdir(data_dir)

train_dir = os.path.join(data_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(data_dir, 'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(data_dir, 'test')
os.mkdir(test_dir)

os.mkdir(csv_dir)


In [4]:
storage_client = storage.Client()
BUCKET_NAME = 'cashcropplants200628'
bucket = storage_client.get_bucket(BUCKET_NAME)
csv_list = list(bucket.list_blobs(prefix='CSV/'))

In [5]:
print(csv_dir)

/home/jupyter/img_csv


In [6]:
for csv_blob in csv_list:
    plant_id = (csv_blob.name[:-4])[4:]
    csv_data = csv_blob.download_as_string()
    plant_df = pd.read_csv(io.BytesIO(csv_data), encoding = 'utf-8', sep = ",")
    plant_df.to_csv('{}/{}.csv'.format(csv_dir, plant_id))

In [7]:
list_csv = os.listdir(csv_dir)

names = []
for csv in list_csv:
    names.append(csv[:-4])

print(len(names))

167


In [8]:
train_dir_list = []
validation_dir_list = []
test_dir_list = []

for name in names:
    train = os.path.join(train_dir, name)
    os.mkdir(train)
    train_dir_list.append(train)
    
    validation = os.path.join(validation_dir, name)
    os.mkdir(validation)
    validation_dir_list.append(validation)
    
    test = os.path.join(test_dir, name)
    os.mkdir(test)
    test_dir_list.append(test)
    
data_dict = dict(zip(names, list(zip(train_dir_list, validation_dir_list, test_dir_list))))

In [9]:
for name in data_dict.keys():
    print('Directories for:', name)
    
    for i in range(len(data_dict[name])):
        print(data_dict[name][i])
        
    print()

Directories for: 16
/home/jupyter/data/train/16
/home/jupyter/data/validation/16
/home/jupyter/data/test/16

Directories for: 2074
/home/jupyter/data/train/2074
/home/jupyter/data/validation/2074
/home/jupyter/data/test/2074

Directories for: 24
/home/jupyter/data/train/24
/home/jupyter/data/validation/24
/home/jupyter/data/test/24

Directories for: 21
/home/jupyter/data/train/21
/home/jupyter/data/validation/21
/home/jupyter/data/test/21

Directories for: 1053
/home/jupyter/data/train/1053
/home/jupyter/data/validation/1053
/home/jupyter/data/test/1053

Directories for: 1012
/home/jupyter/data/train/1012
/home/jupyter/data/validation/1012
/home/jupyter/data/test/1012

Directories for: 1011
/home/jupyter/data/train/1011
/home/jupyter/data/validation/1011
/home/jupyter/data/test/1011

Directories for: 2018
/home/jupyter/data/train/2018
/home/jupyter/data/validation/2018
/home/jupyter/data/test/2018

Directories for: 2077
/home/jupyter/data/train/2077
/home/jupyter/data/validation/2077
/

In [10]:
org_img_dir = 'gs://bucket/cashcropplants200628/img'

In [11]:
def save_blob_jpg_file(x, img_loc, name, set_dir):
    try:
        src = os.path.join(img_loc, name, str(x))
        blob = bucket.blob(src)

        img_str = blob.download_as_string()

        with Image(blob=img_str) as img:
            dst = os.path.join(set_dir, name, '{}.jpg'.format(x))
            img.save(filename=dst)
    except:
        pass

In [ ]:
from __future__ import print_function
from wand.image import Image

train_idx = .8
val_idx = train_idx + .1
img_loc = 'img'

i = 0
for name in data_dict.keys():
    csv_list = list(bucket.list_blobs(prefix='{}/{}/'.format(img_loc, name)))
    file_count = len(csv_list)
    
    print('{} | Number of {}: {}'.format(i, name, file_count))
    
    # Move image data to respective training folder
    print('Moving training images...')
    for x in range(int(train_idx * file_count)):
        x = x + 1
        save_blob_jpg_file(x, img_loc, name, train_dir)

    
    # Move image data to respective validation folder
    print('Moving validation images...')
    for x in range(int(train_idx * file_count), int(val_idx * file_count)):
        x = x + 1
        save_blob_jpg_file(x, img_loc, name, validation_dir)
    
    # Move image data to respective testing folder
    print('Moving testing images...')
    for x in range(int(val_idx * file_count), file_count):
        x = x + 1
        save_blob_jpg_file(x, img_loc, name, test_dir)

        
    i = i + 1

0 | Number of 16: 11
Moving training images...
Moving validation images...
Moving testing images...
1 | Number of 2074: 999
Moving training images...
